In [0]:
import numpy as np
import pandas as pd
import urllib.request, json
from tqdm import tqdm
import cupy as cp
import collections

In [0]:
!pip install --upgrade -q gspread

In [0]:
#авторизация гугл аккуанта
#зайти по ссылке, зайти на аккуант гугла, разрешить доступ и скопировать код в специальную ячейку
#enter
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

In [0]:
#скачивание данных с декларатора
import urllib.request, json 
with urllib.request.urlopen("https://declarator.org/media/dumps/declarations.json") as url:
    data = json.loads(url.read().decode())

In [0]:
class dataset():
    
    def __init__(self, data):
        self.data = data
        #self.clean_data()
        #разделение на женщин, мужчин и тех, у кого не удалось установить пол
        self.not_have_gender, self.male, self.female = self.find_gender()
        
        #все декларации разбиваем по годам в соотвествии с полом
        #для женщин
        self.array_2008_male,self.array_2009_male,self.array_2010_male,self.array_2011_male, \
        self.array_2012_male,self.array_2013_male,\
        self.array_2014_male,self.array_2015_male,self.array_2016_male, \
        self.array_2017_male,self.array_2018_male = self.groups_of_years(self.male)
        
        #для мужчин
        self.array_2008_female,self.array_2009_female,self.array_2010_female,self.array_2011_female, \
        self.array_2012_female,self.array_2013_female,\
        self.array_2014_female,self.array_2015_female,self.array_2016_female, \
        self.array_2017_female,self.array_2018_female = self.groups_of_years(self.female)
        
        
        #заполнение данных
        #id, пол, заработок, кол-во площади, кол-во машин, кол-во недвиж в россии, кол-во недвижимости на западе
        #для женщин
        self.data_array_2008_female = self.fill(self.array_2008_female)
        self.data_array_2009_female = self.fill(self.array_2009_female)
        self.data_array_2010_female = self.fill(self.array_2010_female)
        self.data_array_2011_female = self.fill(self.array_2011_female)
        self.data_array_2012_female = self.fill(self.array_2012_female)
        self.data_array_2013_female = self.fill(self.array_2013_female)
        self.data_array_2014_female = self.fill(self.array_2014_female)
        self.data_array_2015_female = self.fill(self.array_2015_female)
        self.data_array_2016_female = self.fill(self.array_2016_female)
        self.data_array_2017_female = self.fill(self.array_2017_female)
        self.data_array_2018_female = self.fill(self.array_2018_female)
        
        #для мужчин
        self.data_array_2008_male = self.fill(self.array_2008_male)
        self.data_array_2009_male = self.fill(self.array_2009_male)
        self.data_array_2010_male = self.fill(self.array_2010_male)
        self.data_array_2011_male = self.fill(self.array_2011_male)
        self.data_array_2012_male = self.fill(self.array_2012_male)
        self.data_array_2013_male = self.fill(self.array_2013_male)
        self.data_array_2014_male = self.fill(self.array_2014_male)
        self.data_array_2015_male = self.fill(self.array_2015_male)
        self.data_array_2016_male = self.fill(self.array_2016_male)
        self.data_array_2017_male = self.fill(self.array_2017_male)
        self.data_array_2018_male = self.fill(self.array_2018_male)

#     def clean_data(self):
#         for i in range(len(self.data)):
#             #print(i)
#             if 'office' in self.data[i]['main']:
#                 del self.data[i]['main']['office']
#             if len(self.data[i]['real_estates']) > 0 :
#                 if self.data[i]['real_estates'][0]['region'] != None:
#                     if 'name' in self.data[i]['real_estates'][0]['region']:
#                             del self.data[i]['real_estates'][0]['region']['name']
#             if len(self.data[i]['incomes']) > 0 :
#                 if self.data[i]['incomes'][0] != None:
#                     if 'comment' in self.data[i]['incomes'][0]:
#                             del self.data[i]['incomes'][0]['comment']
#             if 'party' in self.data[i]['main']:
#                 del self.data[i]['main']['party']
#             if 'document_type' in self.data[i]['main']:
#                 del self.data[i]['main']['document_type']
    
    #функция поиска гендера
    def find_gender(self):
        not_have_gender = []
        male = []
        female = []
        for i in range(len(self.data)):
            if self.data[i]['main']['person']['gender'] != None:
                if 'M' in self.data[i]['main']['person']['gender']:
                    male.append(self.data[i])
                if 'F' in self.data[i]['main']['person']['gender']:
                    female.append(self.data[i])
            else:
                not_have_gender.append(self.data[i])
        return not_have_gender, male, female
      
    #группировка по годам
    def groups_of_years(self, data):
        array_2008 = []
        array_2009 = []
        array_2010 = []
        array_2011 = []
        array_2012 = []
        array_2013 = []
        array_2014 = []
        array_2015 = []
        array_2016 = []
        array_2017 = []
        array_2018 = []
        for i in range(len(data)):
            if data[i]['main']['year'] == 2008:
                array_2008.append(data[i])
            if data[i]['main']['year'] == 2009:
                array_2009.append(data[i])
            if data[i]['main']['year'] == 2010:
                array_2010.append(data[i])
            if data[i]['main']['year'] == 2011:
                array_2011.append( data[i])
            if data[i]['main']['year'] == 2012:
                array_2012.append( data[i])
            if data[i]['main']['year'] == 2013:
                array_2013.append( data[i])
            if data[i]['main']['year'] == 2014:
                array_2014.append(data[i])
            if data[i]['main']['year'] == 2015:
                array_2015.append( data[i])
            if data[i]['main']['year'] == 2016:
                array_2016.append( data[i])
            if data[i]['main']['year'] == 2017:
                array_2017.append( data[i])
            if data[i]['main']['year'] == 2018:
                array_2018.append( data[i])
        return array_2008,array_2009,array_2010,array_2011,array_2012,array_2013,\
                array_2014,array_2015,array_2016,array_2017,array_2018
     
    #заполнение
    def fill(self, data):
      data_array = []
      for i in range(len(data)):
          
          array = []
          
#           person = data[i]['main']['person']['id']
          
#           gender = data[i]['main']['person']['gender']
          
          inc = 0
          for j in range(len(data[i]['incomes'])):
              if data[i]['incomes'][j]['relative'] != None:
                  inc+= data[i]['incomes'][j]['size']
              if data[i]['incomes'][j]['relative'] == None:
                  inc+= data[i]['incomes'][j]['size']
              


          count = 0
          for data_str in data[i]['real_estates']:
              if data_str['relative'] == None:
                  if data_str['square'] != None:
                      count += data_str['square']
              else:
                  if data_str['square'] != None:
                      count += data_str['square']


          amount = 0
          for data_str in data[i]['vehicles']:
              if data_str['relative'] == None:
                  amount += 1
              else:
                  amount += 1


          rus = 0
          zapad = 0
          for j in range(len(data[i]['real_estates'])):
              if data[i]['real_estates'][j]['relative'] != None:
                  if data[i]['real_estates'][j]['country'] == 'Россия':
                      rus+=1
                  else:
                      zapad +=1
              else:
                  if data[i]['real_estates'][j]['country'] == 'Россия':
                      rus+=1
                  else:
                      zapad +=1
                      
          if inc > 1 and count > 1:
            data_array.append([data[i]['main']['person']['id'], 
                          data[i]['main']['person']['gender'],
                          inc, count, amount, rus, zapad])
                      
      return data_array
    
    #функция сравнения 
    def com(self, row_a, row_b):
      if ((np.absolute( float(row_a[2]) - float(row_b[2]) ) < 2) and
              (np.absolute( float(row_a[3]) - float(row_b[3])) < 2) and
                  (row_a[4] == row_b[4]) and
                      (row_a[5] == row_b[5]) and
                          (row_a[6] == row_b[6])):
          return True
      else:
          return False
    
    
    #главная функция сравнения :)
    def compare(self, d1, d2):
      pairs = []
      for value in tqdm(d1, position=0):
          df_col = []
          #print(len(d2))
          #print('new value = ', value)
          for j in range(len(d2)):
            df_col.append(value)
          #print('len =', len(df_col))
          res_col =  list(map(self.com, df_col, d2))
          #print('new_len =', len(res_col) )
#           #print('first_itr = ', len(res_col))
          for n in range(len(res_col)):
              #print('fffffff')
              if res_col[n] == True:
                pairs.append( [value[0],  d2[n][0]] )
                #print(value[0], 'is similar to', d2[n][0])
      return pairs
    
    #заполнение гугл таблицы
    def pairs_sheet(self, num_work, pairs,year):
      ind_year = sh.get_worksheet(num_work).row_values(1).index(year) +1
      for pair in tqdm(pairs, position = 0):
        if str(pair[0]) in sh.get_worksheet(num_work).col_values(1):
          ind = sh.get_worksheet(num_work).col_values(1).index(str(pair[0]))
          if  sh.get_worksheet(num_work).col_values(2)[ind] == str(pair[1]) :
            sh.get_worksheet(num_work).update_cell(ind+1, ind_year , 1)
          else:
            print('smth goes wrong', pair, 'and', sh.get_worksheet(num_work).col_values(2)[ind])
        else:
          print('there is no pair in this sheet:', pair)
          sh.get_worksheet(num_work).update_cell(len(sh.get_worksheet(num_work).col_values(1))+1, 1, pair[0])
          sh.get_worksheet(num_work).update_cell(len(sh.get_worksheet(num_work).col_values(2))+1, 2, pair[1])
          sh.get_worksheet(num_work).update_cell(sh.get_worksheet(num_work).col_values(1).index(str(pair[0])) +1, ind_year , 1)
          
    #удаление строк в гугл таблице, если нет уверенности по данным, те для всех тех, у кого стоит 1 подтверждение супружества      
    def del_rows(self, num_work):
      n = 2
      l = len(sh.get_worksheet(num_work).col_values(1))
      while n < l:
        #try:
        if sh.get_worksheet(num_work).row_values(n) != []:
          dic = collections.Counter( sh.get_worksheet(num_work).row_values(n) )
          if dic['1'] >= 2:
            n+=1
          else:
            sh.get_worksheet(num_work).delete_row(n)
            print('Del row:', n)
            n -=1
        else:
          break
#         except:
#           gc = gspread.authorize(GoogleCredentials.get_application_default())
#           sh = gc.open('Супружеские пары')
            
            
            

In [0]:
#инициируем класс
dt = dataset(data)

In [97]:
#сравнение и поиск пар
#по необходимости раскоментить и вызвать функцию
#можно раскоментить и все, а можно только те года, которые интересуют
#ничего в данных менять не нужно, просто раскоментить интересующий код и запустить

# pairs_2008 = dt.compare(dt.data_array_2008_male, dt.data_array_2008_female)
# pairs_2009 = dt.compare(dt.data_array_2009_male, dt.data_array_2009_female)
# pairs_2010 = dt.compare(dt.data_array_2010_male, dt.data_array_2010_female)
# pairs_2011 = dt.compare(dt.data_array_2011_male, dt.data_array_2011_female)
# pairs_2012 = dt.compare(dt.data_array_2012_male, dt.data_array_2012_female)
# pairs_2013 = dt.compare(dt.data_array_2013_male, dt.data_array_2013_female)
# pairs_2014 = dt.compare(dt.data_array_2014_male, dt.data_array_2014_female)
# pairs_2015 = dt.compare(dt.data_array_2015_male, dt.data_array_2015_female)
# pairs_2016 = dt.compare(dt.data_array_2016_male, dt.data_array_2016_female)
# pairs_2017 = dt.compare(dt.data_array_2017_male, dt.data_array_2017_female)
# pairs_2018 = dt.compare(dt.data_array_2018_male, dt.data_array_2018_female)

100%|██████████| 2060/2060 [00:02<00:00, 698.28it/s]


In [0]:
#подключаемся к гугл-таблице, название = название существующей таблице в вашем гугл аккуанте
gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.open('Супружеские пары')

In [100]:
#какой год заполнения интересует, такую функцию раскоментить и запустить
#1 = номер листа, который интересует для заполнения. нумерация происходит страниц с нуля!

#dt.pairs_sheet(1, pairs_2008,'2008')
#dt.pairs_sheet(1, pairs_2009,'2009')
#dt.pairs_sheet(1, pairs_2010,'2010')
#dt.pairs_sheet(1, pairs_2011,'2011')
#dt.pairs_sheet(1, pairs_2012,'2012')
#dt.pairs_sheet(1, pairs_2013,'2013')
#dt.pairs_sheet(1, pairs_2014,'2014')
#dt.pairs_sheet(1, pairs_2015,'2015')
#dt.pairs_sheet(1, pairs_2016,'2016')
#dt.pairs_sheet(1, pairs_2017,'2017')
dt.pairs_sheet(1, pairs_2018,'2018')

 20%|██        | 1/5 [00:06<00:24,  6.06s/it]

there is no pair in this sheet: [58670, 58672]


 60%|██████    | 3/5 [00:21<00:13,  6.73s/it]

there is no pair in this sheet: [59230, 59231]


100%|██████████| 5/5 [00:34<00:00,  6.30s/it]


In [25]:
#удаление строк, где кол-во подтверждения супружества под сомнением (то есть все те пары, где супружество установлено только по 1 году)
#передаем в эту функция НОМЕР листа! 
#может выдасть ошибку подключения, в таком случае нужно просто еще раз сделать подключение к гугл таблице и снова вызвать эту функцию

dt.del_rows(2)

Del row: 54
Del row: 54


In [0]:
#ленивая функция, которая сразу показывает разницу между супругами ИЛИ говорит, что декларации в данном году по кому-то отсуствуют
#data_men = массив мужчин с интересующим годом
#data_women = массив женщин с интересующим годом
#number_man = id интересующего мужчины
#number_woman = id интересующей женщины

def find_difference(data_men, data_women, number_man, number_woman):
  x = 0
  y = 0 
  for i in range(len(data_men)):
    if data_men[i][0] == number_man:
      x = data_men[i]

  for i in range(len(data_women)):
    if data_women[i][0] == number_woman:
      y = data_women[i]
      
  if x != 0 and y!=0:
    answer = np.subtract(y[2:], x[2:])
    return answer
  elif x == 0 and y!=0:
    print('Нет данных по женщине')
  elif y == 0 and x!=0:
    print('Нет данных по мужчине')
  if x == 0 and y == 0:
    print('Нет данных по обоим')


In [29]:
#пример использования функции
find_difference(dt.data_array_2014_male, dt.data_array_2014_female, 8493,8504)

Нет данных по обоим
